# 4/4: Brain visualization
By Niloufar Shahdoust (niloufar.shahdoust@utah.edu)

In [1]:
import os
import mat73
import numpy as np
import pandas as pd
import random
from matplotlib import cm
from matplotlib import colormaps
from ast import literal_eval
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from visbrain.objects import BrainObj, SceneObj, SourceObj
from matplotlib.colors import to_hex
from PIL import Image
from sklearn.cluster import KMeans
import matplotlib.colors as mcolors
import seaborn as sns
from fpdf import FPDF
from itertools import cycle
from matplotlib.colors import to_rgba



                * Top view : 'axial_0', 'top'
                * Bottom view : 'axial_1', 'bottom'
                * Left : 'sagittal_0', 'left' view from left
                * Right : 'sagittal_1', 'right' view from right
                * Front : 'coronal_0', 'front'
                * Back : 'coronal_1', 'back'
                * Side front-left : 'side-fl'
                * Side front-right : 'side-fr'
                * Side back-left : 'side-bl'
                * Side back-right : 'side-br'

## Required functions

In [8]:
def get_region_color(region, region_colors):
    if region not in region_colors:
        raise ValueError(f"Region '{region}' not found in colormap.")
    return region_colors[region]



def prepare_visualization_data(df, hemisphere_selected, region_colors):
    """Extract coordinates and corresponding colors from patient data based on the selected hemisphere."""
    
    # Filter the DataFrame based on the selected hemisphere
    if hemisphere_selected == 'left':
        filtered_df = df[df['left'] == 1]
    elif hemisphere_selected == 'right':
        filtered_df = df[df['right'] == 1]
    else:
        raise ValueError("hemisphere_selected must be 'left' or 'right'")
    
    # Extract coordinates
    coordinates = filtered_df[['coordinate_x', 'coordinate_y', 'coordinate_z']].to_numpy()
    
    # Extract colors based on region
    colors = [to_rgba(get_region_color(region, region_colors)) for region in filtered_df['area']]
    
    return coordinates, np.array(colors)




def plot_visbrain(hemisphere_selected, df_list, region_colors, elec_size=20, save=False, output_folder='output', format='pdf'):
    """
    Visualize electrode data on a 3D brain model using Visbrain and save with Matplotlib.

    Args:
        df_list (list): List of DataFrames containing electrode data for patients.
        region_colors (dict): Mapping of brain regions to specific colors.
        elec_size (float): Marker size for electrode contacts.
        save (bool): If True, save the visualization output.
        output_folder (str): Folder to save the output.
        format (str): File format for saving ('pdf' or 'svg').
    """

    os.makedirs(output_folder, exist_ok=True)

    scene_obj = SceneObj(bgcolor='white', size=(10000, 10000))

    # for hemisphere in ['left', 'right']: # Uncomment for both hemispheres
    for hemisphere in ['left']:  # Change this to 'right' if needed.

        brain_obj = BrainObj('B2', translucent=True, hemisphere=hemisphere)
        scene_obj.add_to_subplot(brain_obj, row=0, col=0)
        brain_obj.rotate(fixed='coronal_1')  # Adjust the brain view as needed.

        for df in df_list:
            coordinates, colors = prepare_visualization_data(df, hemisphere_selected, region_colors)
            if hemisphere == hemisphere_selected:
                mask = coordinates[:, 0] < 0  # Left hemisphere
            else:
                mask = coordinates[:, 0] >= 0  # Right hemisphere
            xyz = coordinates[mask]
            filtered_colors = colors[mask]  # Apply the same mask to colors

            if len(xyz) > 0:
                iEEG_obj = SourceObj(
                    name=f'iEEG_{hemisphere}',
                    xyz=xyz,
                    color=filtered_colors,
                    radius_min=elec_size,
                    edge_color=None  # No outer line
                )
                scene_obj.add_to_subplot(iEEG_obj, row=0, col=0)

    if save:
        screenshot_path = os.path.join(output_folder, f'3DModel.{format}')
        try:
            img_array = scene_obj.render()

            # Convert the array to a PIL Image
            img = Image.fromarray(img_array)

            # Use Matplotlib to save the image with advanced styling
            fig, ax = plt.subplots(figsize=(20, 20), dpi=1200)  # Ultra-high resolution
            ax.imshow(img)
            ax.axis('off')  
            plt.tight_layout()

            plt.savefig(screenshot_path, bbox_inches='tight', pad_inches=0, format=format)
            plt.close(fig)
            print(f"3D Brain Model saved at: {screenshot_path}")
        except Exception as e:
            print(f"Error saving the screenshot: {e}")
    else:
        scene_obj.preview()


## reading data

In [3]:
input_folder = '3_brain_visualization_preProcessing_2'
output_folder = '4_brain_visualization_main'


csv_files = [f for f in os.listdir(input_folder) if f.endswith('.csv')]

# Load data into DataFrames
df_patients = []
for file_name in csv_files:
    file_path = os.path.join(input_folder, file_name)
    df = pd.read_csv(file_path)
    df_patients.append(df)

# Gather unique areas across all patients
area_per_patient = [df['area'].unique() for df in df_patients]
area_all_patients = pd.unique(np.concatenate(area_per_patient)).tolist()

# Generate distinct colors using a perceptually uniform colormap
num_colors = len(area_all_patients)
colormap = plt.get_cmap('tab20')  # Use tab20 for up to 20 unique colors
colors_cycle = cycle(colormap(np.linspace(0, 1, num_colors)))

# Convert RGBA to HEX for better compatibility
region_colors = {
    region: mcolors.to_hex(next(colors_cycle)) for region in area_all_patients
}

# Output the region-to-color mapping
print(region_colors)

{'MFG middle frontal gyrus': '#1f77b4', 'MTG middle temporal gyrus': '#ff7f0e', 'ACgG anterior cingulate gyrus': '#2ca02c', 'Hippocampus': '#d62728', 'FuG fusiform gyrus': '#9467bd', 'PHG parahippocampal gyrus': '#8c564b', 'AIns anterior insula': '#e377c2', 'Amygdala': '#7f7f7f', 'SFG superior frontal gyrus': '#bcbd22', 'PP planum polare': '#17becf', 'Caudate': '#9edae5'}


## run

In [9]:
plot_visbrain('left', df_patients, region_colors, elec_size=40, save=True, output_folder=output_folder,  format='pdf')

Creation of a scene
BrainObj(name='B2') created
    BrainObj(name='B2') added to the scene
SourceObj(name='iEEG_left') created
    20 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    18 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    13 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    14 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    14 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    29 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    23 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_left') created
    12 sources detected
    SourceObj(name='iEEG_left') added to the scene
SourceObj(name='iEEG_

3D Brain Model saved at: 4_brain_visualization_main\3DModel.pdf


In [5]:
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()
pdf.set_font("Arial", size=12)

# Add each region and its color to the PDF
for region, color in region_colors.items():
    rgb = tuple(int(color[i:i+2], 16) for i in (1, 3, 5))  # Convert hex to RGB
    pdf.set_text_color(*rgb)  # Set the text color
    pdf.cell(0, 10, region, ln=True)  # Write the text in the region's color


folder_name = "regions.pdf"


complete_output_path = os.path.join(output_folder, folder_name)

pdf.output(complete_output_path)

complete_output_path


'4_brain_visualization_main\\regions.pdf'